In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import numpy as np
import refinitiv.dataplatform as rdp
import eikon as ek
import datetime as dt
from datetime import datetime
import parquet
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.graphics.tsaplots as tsaplots
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_white
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [4]:
df_ti = pd.read_excel('ticker_list_final.xlsx', sheet_name = 'ticker_list_final')
df_sec = pd.read_excel('ticker_list_final.xlsx', sheet_name = 'Sector')

df_ti

,Company,RIC
0,AECOM,ACM.N
1,Imperial Brands,IMB.L
2,Trane Technologies,TT.N
3,DS Smith Plc,SMDS.L
4,Sony Group Corporation,6758.T
...,...,...
120,Logitech International SA,LOGN.S
121,Cognizant Technology Solutions Corp.,CTSH.OQ
122,PostNL,PTNL.AS
123,JetBlue,JBLU.OQ


In [5]:
df_ti.rename(columns={'Company': 'Name'}, inplace=True)
df_ti['Name'][60] = 'Nestlé'
df_ti['Name'][70] = 'LOréal'
df_ti['Name'][80] = 'ARÇELİK A.Ş.'
ti = df_ti.merge(df_sec, on='Name')

ti.drop_duplicates(subset=['Name'], inplace=True)
ti.reset_index(drop=True, inplace=True)

In [6]:
ti_w = ti[['RIC', 'Sector']]

ti_w.index = ti_w['RIC']
ti_w.drop(columns=['RIC'], inplace=True)
ti_w

C:\Users\Evan Mc Garry\AppData\Local\Temp\ipykernel_20112\142337208.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ti_w.drop(columns=['RIC'], inplace=True)


,Sector
RIC,
ACM.N,Industrails
IMB.L,Consumer Staples
TT.N,Industrails
SMDS.L,Materials
6758.T,Technology
...,...
LOGN.S,Information Technology
CTSH.OQ,Information Technology
PTNL.AS,Industrails


In [7]:
ti_w = ti_w.T
ti_w

RIC,ACM.N,IMB.L,TT.N,SMDS.L,6758.T,2308.TW,4544.T,PFE.N,FLS.CO,MNDI.L,...,PANW.OQ,VWS.CO,ORAN.PA,3401.T,VEI.OL,LOGN.S,CTSH.OQ,PTNL.AS,JBLU.OQ,TMO.N
Sector,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care


In [8]:
ti_re = pd.concat([ti_w]*4).reset_index(drop=True)
ti_re

RIC,ACM.N,IMB.L,TT.N,SMDS.L,6758.T,2308.TW,4544.T,PFE.N,FLS.CO,MNDI.L,...,PANW.OQ,VWS.CO,ORAN.PA,3401.T,VEI.OL,LOGN.S,CTSH.OQ,PTNL.AS,JBLU.OQ,TMO.N
0,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care
1,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care
2,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care
3,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care


In [9]:
start_date = datetime.strptime('2018-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2022-01-01', '%Y-%m-%d')

In [10]:
date_list = pd.date_range(start_date, end_date, freq='Y')
dates = pd.DataFrame(date_list, columns=['Dates'])
dates.reset_index(drop=True, inplace=True)
dates

,Dates
0,2018-12-31
1,2019-12-31
2,2020-12-31
3,2021-12-31


In [11]:
ti_re = pd.concat([dates, ti_re], axis=1)
ti_re = ti_re.set_index('Dates')
ti_re

,ACM.N,IMB.L,TT.N,SMDS.L,6758.T,2308.TW,4544.T,PFE.N,FLS.CO,MNDI.L,...,PANW.OQ,VWS.CO,ORAN.PA,3401.T,VEI.OL,LOGN.S,CTSH.OQ,PTNL.AS,JBLU.OQ,TMO.N
Dates,,,,,,,,,,,,,,,,,,,,,
2018-12-31,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care
2019-12-31,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care
2020-12-31,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care
2021-12-31,Industrails,Consumer Staples,Industrails,Materials,Technology,Information Technology,Health Care,Health Care,Industrails,Materials,...,Information Technology,Industrails,Communication Services,Materials,Industrails,Information Technology,Information Technology,Industrails,Industrails,Health Care


In [12]:
ti_panel = pd.DataFrame(ti_re.unstack())
ti_panel.columns=['Sector']
ti_panel

Sector
        Dates                       
ACM.N   2018-12-31       Industrails
        2019-12-31       Industrails
        2020-12-31       Industrails
        2021-12-31       Industrails
IMB.L   2018-12-31  Consumer Staples
...                              ...
JBLU.OQ 2021-12-31       Industrails
TMO.N   2018-12-31       Health Care
        2019-12-31       Health Care
        2020-12-31       Health Care
        2021-12-31       Health Care

[500 rows x 1 columns]

In [13]:
ti_panel.rename_axis(index=['id', 'Dates'], inplace=True)
ti_panel = ti_panel.sort_values(['id', 'Dates'], ascending=[True, True])
ti_panel

Sector
id        Dates                             
002555.SZ 2018-12-31  Communication Services
          2019-12-31  Communication Services
          2020-12-31  Communication Services
          2021-12-31  Communication Services
066570.KS 2018-12-31  Consumer Discretionary
...                                      ...
WSP.TO    2021-12-31             Industrails
YUMC.K    2018-12-31  Consumer Discretionary
          2019-12-31  Consumer Discretionary
          2020-12-31  Consumer Discretionary
          2021-12-31  Consumer Discretionary

[500 rows x 1 columns]

In [14]:
ti_panel.index.get_level_values(0).nunique()

125

In [15]:
ti_panel.index.get_level_values(1).nunique()

4

In [16]:
ti_panel.to_parquet('ti_panel.parquet')

In [17]:
data = pd.read_excel('ticker_list_final.xlsx', sheet_name='FINANCIAL DATA', skiprows=[0,1,2])
data.head()

,Unnamed: 0,ACM.N,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 1116,TMO.N,Unnamed: 1118,Unnamed: 1119,Unnamed: 1120,Unnamed: 1121,Unnamed: 1122,Unnamed: 1123,Unnamed: 1124,Unnamed: 1125
0,NaN,Total Op Exp,Asset TO,EBITDA,Total Op Exp,Deprec and Amort,Quick Rat,Current Rat,ESG S,Total Assets,...,Total Assets,Total Op Exp,Asset TO,EBITDA,Total Op Exp,Deprec and Amort,Quick Rat,Current Rat,ESG S,Total Assets
1,Dates,Total Operating Expense,Asset Turnover,"EBITDA Margin, Percent",Total Operating Expense,Depreciation And Amortization,Quick Ratio,Current Ratio,ESG Score,"Total Assets, Reported",...,"Total Assets, Reported",Total Operating Expense,Asset Turnover,"EBITDA Margin, Percent",Total Operating Expense,Depreciation And Amortization,Quick Ratio,Current Ratio,ESG Score,"Total Assets, Reported"
2,2017-12-31 00:00:00,17549546000,1.297144,4.554325,17549546000,NaN,1.19788,1.19788,48.740255,14396956000,...,9781000000,17950000000,0.407852,25.33225,17950000000,NaN,0.91515,1.33669,70.931193,56669000000
3,2018-12-31 00:00:00,13485713000,0.954555,4.401226,13485713000,NaN,1.16276,1.16276,55.168503,14681131000,...,10959000000,20575000000,0.431493,25.21964,20575000000,NaN,1.23963,1.72849,68.953904,56232000000
4,2019-12-31 00:00:00,13246359000,0.933391,5.312218,13246359000,NaN,1.16604,1.16604,52.745885,14550908000,...,11918000000,21132000000,0.445709,25.597056,21132000000,NaN,1.37534,1.91915,68.799829,58381000000


In [18]:
data.columns = data.columns.to_series().mask(lambda x: x.str.startswith('Unnamed')).ffill()
data = data.T.drop_duplicates().T
data

,NaN,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,IMB.L,...,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N
0,NaN,Total Op Exp,Asset TO,EBITDA,Deprec and Amort,Quick Rat,Current Rat,ESG S,Total Assets,Total Op Exp,...,Current Rat,ESG S,Total Assets,Total Op Exp,Asset TO,EBITDA,Quick Rat,Current Rat,ESG S,Total Assets
1,Dates,Total Operating Expense,Asset Turnover,"EBITDA Margin, Percent",Depreciation And Amortization,Quick Ratio,Current Ratio,ESG Score,"Total Assets, Reported",Total Operating Expense,...,Current Ratio,ESG Score,"Total Assets, Reported",Total Operating Expense,Asset Turnover,"EBITDA Margin, Percent",Quick Ratio,Current Ratio,ESG Score,"Total Assets, Reported"
2,2017-12-31 00:00:00,17549546000,1.297144,4.554325,NaN,1.19788,1.19788,48.740255,14396956000,27969000000,...,0.56198,50.305925,9781000000,17950000000,0.407852,25.33225,0.91515,1.33669,70.931193,56669000000
3,2018-12-31 00:00:00,13485713000,0.954555,4.401226,NaN,1.16276,1.16276,55.168503,14681131000,27659000000,...,0.5497,53.125946,10959000000,20575000000,0.431493,25.21964,1.23963,1.72849,68.953904,56232000000
4,2019-12-31 00:00:00,13246359000,0.933391,5.312218,NaN,1.16604,1.16604,52.745885,14550908000,29397000000,...,0.67067,49.3894,11918000000,21132000000,0.445709,25.597056,1.37534,1.91915,68.799829,58381000000
5,2020-12-31 00:00:00,12858515000,0.961165,5.728115,NaN,1.23642,1.23642,51.236172,12998951000,29831000000,...,1.25093,52.583528,13406000000,24424000000,0.505646,31.650009,1.73991,2.13092,68.770176,69052000000
6,2021-12-31 00:00:00,12711299000,1.078794,6.144653,NaN,1.11805,1.11805,NaN,11733954000,29645000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
data = data.drop([0]).reset_index(drop=True)

In [20]:
data.columns.values[0] = 'Dates'

In [21]:
df = data.copy()

In [22]:
df.drop(['Dates'], axis=1, inplace=True)
df.head()

,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,...,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N
0,Total Operating Expense,Asset Turnover,"EBITDA Margin, Percent",Depreciation And Amortization,Quick Ratio,Current Ratio,ESG Score,"Total Assets, Reported",Total Operating Expense,Asset Turnover,...,Current Ratio,ESG Score,"Total Assets, Reported",Total Operating Expense,Asset Turnover,"EBITDA Margin, Percent",Quick Ratio,Current Ratio,ESG Score,"Total Assets, Reported"
1,17549546000,1.297144,4.554325,NaN,1.19788,1.19788,48.740255,14396956000,27969000000,0.949387,...,0.56198,50.305925,9781000000,17950000000,0.407852,25.33225,0.91515,1.33669,70.931193,56669000000
2,13485713000,0.954555,4.401226,NaN,1.16276,1.16276,55.168503,14681131000,27659000000,0.972412,...,0.5497,53.125946,10959000000,20575000000,0.431493,25.21964,1.23963,1.72849,68.953904,56232000000
3,13246359000,0.933391,5.312218,NaN,1.16604,1.16604,52.745885,14550908000,29397000000,0.988517,...,0.67067,49.3894,11918000000,21132000000,0.445709,25.597056,1.37534,1.91915,68.799829,58381000000
4,12858515000,0.961165,5.728115,NaN,1.23642,1.23642,51.236172,12998951000,29831000000,0.996023,...,1.25093,52.583528,13406000000,24424000000,0.505646,31.650009,1.73991,2.13092,68.770176,69052000000


In [23]:
muls_cols = pd.MultiIndex.from_arrays([df.columns, df.iloc[0].values])
muls_cols

MultiIndex([(  'ACM.N',       'Total Operating Expense'),
            (  'ACM.N',                'Asset Turnover'),
            (  'ACM.N',        'EBITDA Margin, Percent'),
            (  'ACM.N', 'Depreciation And Amortization'),
            (  'ACM.N',                   'Quick Ratio'),
            (  'ACM.N',                 'Current Ratio'),
            (  'ACM.N',                     'ESG Score'),
            (  'ACM.N',        'Total Assets, Reported'),
            (  'IMB.L',       'Total Operating Expense'),
            (  'IMB.L',                'Asset Turnover'),
            ...
            ('JBLU.OQ',                 'Current Ratio'),
            ('JBLU.OQ',                     'ESG Score'),
            ('JBLU.OQ',        'Total Assets, Reported'),
            (  'TMO.N',       'Total Operating Expense'),
            (  'TMO.N',                'Asset Turnover'),
            (  'TMO.N',        'EBITDA Margin, Percent'),
            (  'TMO.N',                   'Quick Ratio')

In [24]:
data=data.drop([0]).reset_index(drop=True)
data

,Dates,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,IMB.L,...,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N
0,2017-12-31 00:00:00,17549546000,1.297144,4.554325,NaN,1.19788,1.19788,48.740255,14396956000,27969000000,...,0.56198,50.305925,9781000000,17950000000,0.407852,25.33225,0.91515,1.33669,70.931193,56669000000
1,2018-12-31 00:00:00,13485713000,0.954555,4.401226,NaN,1.16276,1.16276,55.168503,14681131000,27659000000,...,0.5497,53.125946,10959000000,20575000000,0.431493,25.21964,1.23963,1.72849,68.953904,56232000000
2,2019-12-31 00:00:00,13246359000,0.933391,5.312218,NaN,1.16604,1.16604,52.745885,14550908000,29397000000,...,0.67067,49.3894,11918000000,21132000000,0.445709,25.597056,1.37534,1.91915,68.799829,58381000000
3,2020-12-31 00:00:00,12858515000,0.961165,5.728115,NaN,1.23642,1.23642,51.236172,12998951000,29831000000,...,1.25093,52.583528,13406000000,24424000000,0.505646,31.650009,1.73991,2.13092,68.770176,69052000000
4,2021-12-31 00:00:00,12711299000,1.078794,6.144653,NaN,1.11805,1.11805,NaN,11733954000,29645000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
data['Dates'] = pd.to_datetime(data['Dates'])

In [26]:
data.set_index(data['Dates'], inplace=True)
data.drop(['Dates'], axis=1, inplace=True)
data

,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,...,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N,TMO.N
Dates,,,,,,,,,,,,,,,,,,,,,
2017-12-31,17549546000,1.297144,4.554325,NaN,1.19788,1.19788,48.740255,14396956000,27969000000,0.949387,...,0.56198,50.305925,9781000000,17950000000,0.407852,25.33225,0.91515,1.33669,70.931193,56669000000
2018-12-31,13485713000,0.954555,4.401226,NaN,1.16276,1.16276,55.168503,14681131000,27659000000,0.972412,...,0.5497,53.125946,10959000000,20575000000,0.431493,25.21964,1.23963,1.72849,68.953904,56232000000
2019-12-31,13246359000,0.933391,5.312218,NaN,1.16604,1.16604,52.745885,14550908000,29397000000,0.988517,...,0.67067,49.3894,11918000000,21132000000,0.445709,25.597056,1.37534,1.91915,68.799829,58381000000
2020-12-31,12858515000,0.961165,5.728115,NaN,1.23642,1.23642,51.236172,12998951000,29831000000,0.996023,...,1.25093,52.583528,13406000000,24424000000,0.505646,31.650009,1.73991,2.13092,68.770176,69052000000
2021-12-31,12711299000,1.078794,6.144653,NaN,1.11805,1.11805,NaN,11733954000,29645000000,1.068111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
data.columns = muls_cols
data.head()

ACM.N                                        \
           Total Operating Expense Asset Turnover EBITDA Margin, Percent   
Dates                                                                      
2017-12-31             17549546000       1.297144               4.554325   
2018-12-31             13485713000       0.954555               4.401226   
2019-12-31             13246359000       0.933391               5.312218   
2020-12-31             12858515000       0.961165               5.728115   
2021-12-31             12711299000       1.078794               6.144653   

                                                                               \
           Depreciation And Amortization Quick Ratio Current Ratio  ESG Score   
Dates                                                                           
2017-12-31                           NaN     1.19788       1.19788  48.740255   
2018-12-31                           NaN     1.16276       1.16276  55.168503   
2019-12-31                           NaN     1.16604       1.16604  52.745885   
2020-12-31                           NaN     1.23642       1.23642  51.236172   
2021-12-31                           NaN     1.11805       1.11805        NaN   

                                                    IMB.L                 ...  \
           Total Assets, Reported Total Operating Expense Asset Turnover  ...   
Dates                                                                     ...   
2017-12-31            14396956000             27969000000       0.949387  ...   
2018-12-31            14681131000             27659000000       0.972412  ...   
2019-12-31            14550908000             29397000000       0.988517  ...   
2020-12-31            12998951000             29831000000       0.996023  ...   
2021-12-31            11733954000             29645000000       1.068111  ...   

                 JBLU.OQ                                    \
           Current Ratio  ESG Score Total Assets, Reported   
Dates                                                        
2017-12-31       0.56198  50.305925             9781000000   
2018-12-31        0.5497  53.125946            10959000000   
2019-12-31       0.67067    49.3894            11918000000   
2020-12-31       1.25093  52.583528            13406000000   
2021-12-31           NaN        NaN                    NaN   

                             TMO.N                                        \
           Total Operating Expense Asset Turnover EBITDA Margin, Percent   
Dates                                                                      
2017-12-31             17950000000       0.407852               25.33225   
2018-12-31             20575000000       0.431493               25.21964   
2019-12-31             21132000000       0.445709              25.597056   
2020-12-31             24424000000       0.505646              31.650009   
2021-12-31                     NaN            NaN                    NaN   

                                                                        
           Quick Ratio Current Ratio  ESG Score Total Assets, Reported  
Dates                                                                   
2017-12-31     0.91515       1.33669  70.931193            56669000000  
2018-12-31     1.23963       1.72849  68.953904            56232000000  
2019-12-31     1.37534       1.91915  68.799829            58381000000  
2020-12-31     1.73991       2.13092  68.770176            69052000000  
2021-12-31         NaN           NaN        NaN                    NaN  

[5 rows x 961 columns]

In [28]:
data.index

DatetimeIndex(['2017-12-31', '2018-12-31', '2019-12-31', '2020-12-31',
               '2021-12-31'],
              dtype='datetime64[ns]', name='Dates', freq=None)

In [29]:
data.index = data.index + pd.tseries.offsets.YearEnd()
data.index

DatetimeIndex(['2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31',
               '2022-12-31'],
              dtype='datetime64[ns]', name='Dates', freq=None)

In [30]:
data = data[:'2021-12-31']
data

ACM.N                                        \
           Total Operating Expense Asset Turnover EBITDA Margin, Percent   
Dates                                                                      
2018-12-31             17549546000       1.297144               4.554325   
2019-12-31             13485713000       0.954555               4.401226   
2020-12-31             13246359000       0.933391               5.312218   
2021-12-31             12858515000       0.961165               5.728115   

                                                                               \
           Depreciation And Amortization Quick Ratio Current Ratio  ESG Score   
Dates                                                                           
2018-12-31                           NaN     1.19788       1.19788  48.740255   
2019-12-31                           NaN     1.16276       1.16276  55.168503   
2020-12-31                           NaN     1.16604       1.16604  52.745885   
2021-12-31                           NaN     1.23642       1.23642  51.236172   

                                                    IMB.L                 ...  \
           Total Assets, Reported Total Operating Expense Asset Turnover  ...   
Dates                                                                     ...   
2018-12-31            14396956000             27969000000       0.949387  ...   
2019-12-31            14681131000             27659000000       0.972412  ...   
2020-12-31            14550908000             29397000000       0.988517  ...   
2021-12-31            12998951000             29831000000       0.996023  ...   

                 JBLU.OQ                                    \
           Current Ratio  ESG Score Total Assets, Reported   
Dates                                                        
2018-12-31       0.56198  50.305925             9781000000   
2019-12-31        0.5497  53.125946            10959000000   
2020-12-31       0.67067    49.3894            11918000000   
2021-12-31       1.25093  52.583528            13406000000   

                             TMO.N                                        \
           Total Operating Expense Asset Turnover EBITDA Margin, Percent   
Dates                                                                      
2018-12-31             17950000000       0.407852               25.33225   
2019-12-31             20575000000       0.431493               25.21964   
2020-12-31             21132000000       0.445709              25.597056   
2021-12-31             24424000000       0.505646              31.650009   

                                                                        
           Quick Ratio Current Ratio  ESG Score Total Assets, Reported  
Dates                                                                   
2018-12-31     0.91515       1.33669  70.931193            56669000000  
2019-12-31     1.23963       1.72849  68.953904            56232000000  
2020-12-31     1.37534       1.91915  68.799829            58381000000  
2021-12-31     1.73991       2.13092  68.770176            69052000000  

[4 rows x 961 columns]

In [31]:
data = data[~data.index.duplicated()]
data

ACM.N                                        \
           Total Operating Expense Asset Turnover EBITDA Margin, Percent   
Dates                                                                      
2018-12-31             17549546000       1.297144               4.554325   
2019-12-31             13485713000       0.954555               4.401226   
2020-12-31             13246359000       0.933391               5.312218   
2021-12-31             12858515000       0.961165               5.728115   

                                                                               \
           Depreciation And Amortization Quick Ratio Current Ratio  ESG Score   
Dates                                                                           
2018-12-31                           NaN     1.19788       1.19788  48.740255   
2019-12-31                           NaN     1.16276       1.16276  55.168503   
2020-12-31                           NaN     1.16604       1.16604  52.745885   
2021-12-31                           NaN     1.23642       1.23642  51.236172   

                                                    IMB.L                 ...  \
           Total Assets, Reported Total Operating Expense Asset Turnover  ...   
Dates                                                                     ...   
2018-12-31            14396956000             27969000000       0.949387  ...   
2019-12-31            14681131000             27659000000       0.972412  ...   
2020-12-31            14550908000             29397000000       0.988517  ...   
2021-12-31            12998951000             29831000000       0.996023  ...   

                 JBLU.OQ                                    \
           Current Ratio  ESG Score Total Assets, Reported   
Dates                                                        
2018-12-31       0.56198  50.305925             9781000000   
2019-12-31        0.5497  53.125946            10959000000   
2020-12-31       0.67067    49.3894            11918000000   
2021-12-31       1.25093  52.583528            13406000000   

                             TMO.N                                        \
           Total Operating Expense Asset Turnover EBITDA Margin, Percent   
Dates                                                                      
2018-12-31             17950000000       0.407852               25.33225   
2019-12-31             20575000000       0.431493               25.21964   
2020-12-31             21132000000       0.445709              25.597056   
2021-12-31             24424000000       0.505646              31.650009   

                                                                        
           Quick Ratio Current Ratio  ESG Score Total Assets, Reported  
Dates                                                                   
2018-12-31     0.91515       1.33669  70.931193            56669000000  
2019-12-31     1.23963       1.72849  68.953904            56232000000  
2020-12-31     1.37534       1.91915  68.799829            58381000000  
2021-12-31     1.73991       2.13092  68.770176            69052000000  

[4 rows x 961 columns]

In [32]:
fin_panel = data.stack(level=0, dropna=False)
fin_panel.head()

Asset Turnover Current Ratio  \
Dates                                               
2018-12-31 002555.SZ       0.796177       2.60977   
           066570.KS       1.552838       1.09457   
           2232.HK         1.368711       1.18821   
           2308.TW         0.920729       1.65548   
           2587.T          0.838484       0.95106   

                     Depreciation And Amortization EBITDA Margin, Percent  \
Dates                                                                       
2018-12-31 002555.SZ                      16270790              27.457111   
           066570.KS                  426854000000               6.899633   
           2232.HK                         4917000               12.13254   
           2308.TW                      1870253000              13.321294   
           2587.T                      35881000000              15.311975   

                      ESG Score Quick Ratio Total Assets, Reported  \
Dates                                                                
2018-12-31 002555.SZ  27.347583     2.54271             9160137680   
           066570.KS  92.720554     0.75765         41220959000000   
           2232.HK          NaN     0.89397             1821725000   
           2308.TW    75.379627     1.30999           250538559000   
           2587.T        57.232     0.77225          1522029000000   

                     Total Operating Expense  
Dates                                         
2018-12-31 002555.SZ              4671079410  
           066570.KS          59022645000000  
           2232.HK                1984389000  
           2308.TW              204466909000  
           2587.T              1116100000000

In [33]:
fin_panel.rename_axis(index=['Dates', 'id'],inplace=True)

fin_panel=fin_panel.swaplevel()

fin_panel=fin_panel.sort_values(['id', 'Dates'], ascending=[True, True])

fin_panel=fin_panel.astype(float)
fin_panel.head()

Asset Turnover  Current Ratio  \
id        Dates                                       
002555.SZ 2018-12-31        0.796177        2.60977   
          2019-12-31        0.869529        2.09920   
          2020-12-31        1.426797        2.44392   
          2021-12-31        1.390631        1.28937   
066570.KS 2018-12-31        1.552838        1.09457   

                      Depreciation And Amortization  EBITDA Margin, Percent  \
id        Dates                                                               
002555.SZ 2018-12-31                   1.627079e+07               27.457111   
          2019-12-31                   2.717638e+07               23.891154   
          2020-12-31                   1.589739e+07               20.995620   
          2021-12-31                   6.806198e+07               21.787496   
066570.KS 2018-12-31                   4.268540e+11                6.899633   

                      ESG Score  Quick Ratio  Total Assets, Reported  \
id        Dates                                                        
002555.SZ 2018-12-31  27.347583      2.54271            9.160138e+09   
          2019-12-31  38.609209      2.09920            8.395762e+09   
          2020-12-31  50.547773      2.44392            1.014526e+10   
          2021-12-31  59.321713      1.28937            1.056433e+10   
066570.KS 2018-12-31  92.720554      0.75765            4.122096e+13   

                      Total Operating Expense  
id        Dates                                
002555.SZ 2018-12-31             4.671079e+09  
          2019-12-31             6.863570e+09  
          2020-12-31             1.055489e+10  
          2021-12-31             1.114381e+10  
066570.KS 2018-12-31             5.902264e+13

In [34]:
fin_panel.index.get_level_values(0).nunique()

125

In [35]:
fin_panel.index.get_level_values(1).nunique()

4

In [36]:
emission = pd.read_excel('ticker_list_final.xlsx', sheet_name='EMISSIONS', skiprows=[0,1,2])
emission.head()

,Unnamed: 0,ACM.N,Unnamed: 2,Unnamed: 3,IMB.L,Unnamed: 5,Unnamed: 6,TT.N,Unnamed: 8,Unnamed: 9,...,Unnamed: 366,PTNL.AS,Unnamed: 368,Unnamed: 369,JBLU.OQ,Unnamed: 371,Unnamed: 372,TMO.N,Unnamed: 374,Unnamed: 375
0,Dates,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,...,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location
1,2018-12-31 00:00:00,-,-,-,108241,155648,150348,343565,191786,192939,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
2,2019-12-31 00:00:00,-,-,-,99577,131737,147039,304585,141753,154027,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
3,2020-12-31 00:00:00,23149,-,70870,95987,136401,143390,273621,75149,139193,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
4,2021-12-31 00:00:00,19689,-,53796,91129,85476,132550,257632,61086,137264,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [37]:
emission.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

emission.columns = emission.columns.to_series().mask(lambda x: x.str.startswith('Unnamed')).ffill()
emission.head()

,Date,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,IMB.L,TT.N,TT.N,TT.N,...,CTSH.OQ,PTNL.AS,PTNL.AS,PTNL.AS,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N
0,Dates,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,...,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location
1,2018-12-31 00:00:00,-,-,-,108241,155648,150348,343565,191786,192939,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
2,2019-12-31 00:00:00,-,-,-,99577,131737,147039,304585,141753,154027,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
3,2020-12-31 00:00:00,23149,-,70870,95987,136401,143390,273621,75149,139193,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
4,2021-12-31 00:00:00,19689,-,53796,91129,85476,132550,257632,61086,137264,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [38]:
df2=emission.copy()
df2['Date'][1].strftime('%Y-%m-%d')
df2
# pd.to_datetime(df2['Date'][1:], format='%Y-%m-%d')
# df2.head()

,Date,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,IMB.L,TT.N,TT.N,TT.N,...,CTSH.OQ,PTNL.AS,PTNL.AS,PTNL.AS,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N
0,Dates,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,...,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location
1,2018-12-31 00:00:00,-,-,-,108241,155648,150348,343565,191786,192939,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
2,2019-12-31 00:00:00,-,-,-,99577,131737,147039,304585,141753,154027,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
3,2020-12-31 00:00:00,23149,-,70870,95987,136401,143390,273621,75149,139193,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
4,2021-12-31 00:00:00,19689,-,53796,91129,85476,132550,257632,61086,137264,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [39]:
df2.columns.values[0] = 'Dates'
df2.head()

,Dates,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,IMB.L,TT.N,TT.N,TT.N,...,CTSH.OQ,PTNL.AS,PTNL.AS,PTNL.AS,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N
0,Dates,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,...,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location
1,2018-12-31 00:00:00,-,-,-,108241,155648,150348,343565,191786,192939,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
2,2019-12-31 00:00:00,-,-,-,99577,131737,147039,304585,141753,154027,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
3,2020-12-31 00:00:00,23149,-,70870,95987,136401,143390,273621,75149,139193,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
4,2021-12-31 00:00:00,19689,-,53796,91129,85476,132550,257632,61086,137264,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [40]:
df2.drop(['Dates'],axis=1,inplace=True)
mul_cols=pd.MultiIndex.from_arrays([df2.columns, df2.iloc[0].values])
mul_cols

MultiIndex([(  'ACM.N',          'Scope 1'),
            (  'ACM.N',   'Scope 2 Market'),
            (  'ACM.N', 'Scope 2 Location'),
            (  'IMB.L',          'Scope 1'),
            (  'IMB.L',   'Scope 2 Market'),
            (  'IMB.L', 'Scope 2 Location'),
            (   'TT.N',          'Scope 1'),
            (   'TT.N',   'Scope 2 Market'),
            (   'TT.N', 'Scope 2 Location'),
            ( 'SMDS.L',          'Scope 1'),
            ...
            ('CTSH.OQ', 'Scope 2 Location'),
            ('PTNL.AS',          'Scope 1'),
            ('PTNL.AS',   'Scope 2 Market'),
            ('PTNL.AS', 'Scope 2 Location'),
            ('JBLU.OQ',          'Scope 1'),
            ('JBLU.OQ',   'Scope 2 Market'),
            ('JBLU.OQ', 'Scope 2 Location'),
            (  'TMO.N',          'Scope 1'),
            (  'TMO.N',   'Scope 2 Market'),
            (  'TMO.N', 'Scope 2 Location')],
           length=375)

In [41]:
emission.head()

,Dates,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,IMB.L,TT.N,TT.N,TT.N,...,CTSH.OQ,PTNL.AS,PTNL.AS,PTNL.AS,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N
0,Dates,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,...,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location,Scope 1,Scope 2 Market,Scope 2 Location
1,2018-12-31 00:00:00,-,-,-,108241,155648,150348,343565,191786,192939,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
2,2019-12-31 00:00:00,-,-,-,99577,131737,147039,304585,141753,154027,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
3,2020-12-31 00:00:00,23149,-,70870,95987,136401,143390,273621,75149,139193,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
4,2021-12-31 00:00:00,19689,-,53796,91129,85476,132550,257632,61086,137264,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [42]:
emission=emission.drop([0]).reset_index(drop=True)
emission.head()

,Dates,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,IMB.L,TT.N,TT.N,TT.N,...,CTSH.OQ,PTNL.AS,PTNL.AS,PTNL.AS,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N
0,2018-12-31 00:00:00,-,-,-,108241,155648,150348,343565,191786,192939,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
1,2019-12-31 00:00:00,-,-,-,99577,131737,147039,304585,141753,154027,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
2,2020-12-31 00:00:00,23149,-,70870,95987,136401,143390,273621,75149,139193,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
3,2021-12-31 00:00:00,19689,-,53796,91129,85476,132550,257632,61086,137264,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [43]:
emission['Dates']=pd.to_datetime(emission['Dates'])
emission.set_index(emission['Dates'],inplace=True)
emission.head()

,Dates,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,IMB.L,TT.N,TT.N,TT.N,...,CTSH.OQ,PTNL.AS,PTNL.AS,PTNL.AS,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N
Dates,,,,,,,,,,,,,,,,,,,,,
2018-12-31,2018-12-31,-,-,-,108241,155648,150348,343565,191786,192939,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
2019-12-31,2019-12-31,-,-,-,99577,131737,147039,304585,141753,154027,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
2020-12-31,2020-12-31,23149,-,70870,95987,136401,143390,273621,75149,139193,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
2021-12-31,2021-12-31,19689,-,53796,91129,85476,132550,257632,61086,137264,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [44]:
emission.drop(['Dates'],axis=1,inplace=True) 
emission.head()

,ACM.N,ACM.N,ACM.N,IMB.L,IMB.L,IMB.L,TT.N,TT.N,TT.N,SMDS.L,...,CTSH.OQ,PTNL.AS,PTNL.AS,PTNL.AS,JBLU.OQ,JBLU.OQ,JBLU.OQ,TMO.N,TMO.N,TMO.N
Dates,,,,,,,,,,,,,,,,,,,,,
2018-12-31,-,-,-,108241,155648,150348,343565,191786,192939,1750321,...,-,37031,-,442,8370944,-,26656,283161,416195,373648
2019-12-31,-,-,-,99577,131737,147039,304585,141753,154027,2400758,...,-,40491,-,221,8730756,-,25944,280407,408555,339090
2020-12-31,23149,-,70870,95987,136401,143390,273621,75149,139193,2047265,...,148138,40368,-,174,4063538,-,23758,278993,334885,345364
2021-12-31,19689,-,53796,91129,85476,132550,257632,61086,137264,2023278,...,97882,34288,-,171,6853927,-,25945,298043,314972,365356


In [45]:
emission.columns=mul_cols
emission.head()

ACM.N                                   IMB.L                 \
           Scope 1 Scope 2 Market Scope 2 Location Scope 1 Scope 2 Market   
Dates                                                                       
2018-12-31       -              -                -  108241         155648   
2019-12-31       -              -                -   99577         131737   
2020-12-31   23149              -            70870   95987         136401   
2021-12-31   19689              -            53796   91129          85476   

                               TT.N                                   SMDS.L  \
           Scope 2 Location Scope 1 Scope 2 Market Scope 2 Location  Scope 1   
Dates                                                                          
2018-12-31           150348  343565         191786           192939  1750321   
2019-12-31           147039  304585         141753           154027  2400758   
2020-12-31           143390  273621          75149           139193  2047265   
2021-12-31           132550  257632          61086           137264  2023278   

            ...          CTSH.OQ PTNL.AS                                  \
            ... Scope 2 Location Scope 1 Scope 2 Market Scope 2 Location   
Dates       ...                                                            
2018-12-31  ...                -   37031              -              442   
2019-12-31  ...                -   40491              -              221   
2020-12-31  ...           148138   40368              -              174   
2021-12-31  ...            97882   34288              -              171   

            JBLU.OQ                                   TMO.N                 \
            Scope 1 Scope 2 Market Scope 2 Location Scope 1 Scope 2 Market   
Dates                                                                        
2018-12-31  8370944              -            26656  283161         416195   
2019-12-31  8730756              -            25944  280407         408555   
2020-12-31  4063538              -            23758  278993         334885   
2021-12-31  6853927              -            25945  298043         314972   

                             
           Scope 2 Location  
Dates                        
2018-12-31           373648  
2019-12-31           339090  
2020-12-31           345364  
2021-12-31           365356  

[4 rows x 375 columns]

In [46]:
emission_panel=emission.stack(0,dropna=False)
emission_panel.head()

Scope 1 Scope 2 Location Scope 2 Market
Dates                                                       
2018-12-31 002555.SZ       -                -              -
           066570.KS  785312           846510           5226
           2232.HK         -                -              -
           2308.TW     25670           370846         298159
           2587.T     219000           205000              -

In [47]:
emission_panel.rename_axis(index=['Dates', 'id'],inplace=True)

emission_panel=emission_panel.swaplevel()

emission_panel=emission_panel.sort_values(['id', 'Dates'], ascending=[True, True])

emission_panel.head()

Scope 1 Scope 2 Location Scope 2 Market
id        Dates                                             
002555.SZ 2018-12-31       -                -              -
          2019-12-31     129             2701              -
          2020-12-31     169             2395              -
          2021-12-31     152             1794              -
066570.KS 2018-12-31  785312           846510           5226

In [48]:
emission_panel.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 500 entries, ('002555.SZ', Timestamp('2018-12-31 00:00:00')) to ('YUMC.K', Timestamp('2021-12-31 00:00:00'))
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Scope 1           500 non-null    object
 1   Scope 2 Location  500 non-null    object
 2   Scope 2 Market    500 non-null    object
dtypes: object(3)
memory usage: 18.0+ KB


In [49]:
emission_panel.replace('-', np.nan, inplace=True)

emission_panel=emission_panel.astype(float)
emission_panel.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 500 entries, ('002555.SZ', Timestamp('2018-12-31 00:00:00')) to ('YUMC.K', Timestamp('2021-12-31 00:00:00'))
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Scope 1           439 non-null    float64
 1   Scope 2 Location  420 non-null    float64
 2   Scope 2 Market    344 non-null    float64
dtypes: float64(3)
memory usage: 18.0+ KB


In [50]:
emission_panel.index.get_level_values(0).nunique()

125

In [51]:
emission_panel.index.get_level_values(1).nunique()

4

In [52]:
ti_panel=ti_panel.sort_values(['id', 'Dates'], ascending=[True, True])
fin_panel=fin_panel.sort_values(['id', 'Dates'], ascending=[True, True])
emission_panel=emission_panel.sort_values(['id', 'Dates'], ascending=[True, True])

In [53]:
panel=pd.concat([ti_panel,fin_panel,emission_panel], axis=1)
panel=panel.sort_values(['id', 'Dates'], ascending=[True, True])
panel.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 500 entries, ('002555.SZ', Timestamp('2018-12-31 00:00:00')) to ('YUMC.K', Timestamp('2021-12-31 00:00:00'))
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Sector                         500 non-null    object 
 1   Asset Turnover                 490 non-null    float64
 2   Current Ratio                  495 non-null    float64
 3   Depreciation And Amortization  337 non-null    float64
 4   EBITDA Margin, Percent         496 non-null    float64
 5   ESG Score                      468 non-null    float64
 6   Quick Ratio                    495 non-null    float64
 7   Total Assets, Reported         491 non-null    float64
 8   Total Operating Expense        496 non-null    float64
 9   Scope 1                        439 non-null    float64
 10  Scope 2 Location               420 non-null    float64
 11  Scope 2 Market         

In [54]:
panel.index.get_level_values(0).nunique()

125

In [55]:
panel.index.get_level_values(1).nunique()

4

In [56]:
# Rename Index from id to ticker
panel.index.names=['ticker', 'Dates']

# Backfill and Forwardfill missing values
panel=panel.groupby(level=0).apply(lambda x: x.fillna(method='bfill'))
panel=panel.groupby(level=0).apply(lambda x: x.fillna(method='ffill'))

# Replace NaN Values in Scope 2 Market Based with Scope 2 Location Based
panel['Scope 2 Market']=panel['Scope 2 Market'].fillna(panel['Scope 2 Location'])

# Drop All Data from Consumer Nonyclicals
panel=panel.drop(panel.loc[panel['Sector']=='Consumer Noncyclicals'].index)

# Drop Depreciation and Amortization
panel=panel.drop(['Depreciation And Amortization'], axis=1)


In [57]:
# Compute ESG Score Mean for each Sector
panel['ESG Score']=panel.groupby('Sector')['ESG Score'].transform(lambda x: x.fillna(x.mean()))

# Input ESG Mean for Consumer Staples into Consumer Cyclicals
df_cs = panel.loc[panel['Sector']=='Consumer Staples']
mu = df_cs['ESG Score'].mean()
panel['ESG Score']=panel['ESG Score'].fillna(mu)

# Input Total Assets, Reported for Utilities into Utilities
df_ut = panel.loc[panel['Sector']=='Utilities']
df_ut2 = df_ut.loc[df_ut['Total Assets, Reported'].notnull()]
mu = df_ut2['Total Assets, Reported'].mean()
panel['Total Assets, Reported']=panel['Total Assets, Reported'].fillna(mu)

# Compute Scope 1 Mean for each Sector
panel['Scope 1']=panel.groupby('Sector')['Scope 1'].transform(lambda x: x.fillna(x.mean()))

# Compute Scope 2-L Mean for each Sector
panel['Scope 2 Location']=panel.groupby('Sector')['Scope 2 Location'].transform(lambda x: x.fillna(x.mean()))

# Compute Scope 2-M Mean for each Sector
panel['Scope 2 Market']=panel.groupby('Sector')['Scope 2 Market'].transform(lambda x: x.fillna(x.mean()))

# Check for Missing Values
cols=panel.columns.tolist()
for col in cols:
     print(col)
     print(panel[col].isnull().sum())

Sector
0
Asset Turnover
0
Current Ratio
0
EBITDA Margin, Percent
0
ESG Score
0
Quick Ratio
0
Total Assets, Reported
0
Total Operating Expense
0
Scope 1
0
Scope 2 Location
0
Scope 2 Market
0


In [58]:
# Create Expense Ratio
panel['Expense Ratio']=panel['Total Operating Expense']/panel['Total Assets, Reported']
panel.drop(['Total Operating Expense', 'Total Assets, Reported'], axis=1, inplace=True)
panel

Sector  Asset Turnover  Current Ratio  \
ticker    Dates                                                               
002555.SZ 2018-12-31  Communication Services        0.796177        2.60977   
          2019-12-31  Communication Services        0.869529        2.09920   
          2020-12-31  Communication Services        1.426797        2.44392   
          2021-12-31  Communication Services        1.390631        1.28937   
066570.KS 2018-12-31  Consumer Discretionary        1.552838        1.09457   
...                                      ...             ...            ...   
WSP.TO    2021-12-31             Industrails        1.005384        1.03480   
YUMC.K    2018-12-31  Consumer Discretionary        1.938857        1.75977   
          2019-12-31  Consumer Discretionary        1.891649        1.55786   
          2020-12-31  Consumer Discretionary        1.518339        1.30127   
          2021-12-31  Consumer Discretionary        0.927125        2.38800   

                      EBITDA Margin, Percent  ESG Score  Quick Ratio  \
ticker    Dates                                                        
002555.SZ 2018-12-31               27.457111  27.347583      2.54271   
          2019-12-31               23.891154  38.609209      2.09920   
          2020-12-31               20.995620  50.547773      2.44392   
          2021-12-31               21.787496  59.321713      1.28937   
066570.KS 2018-12-31                6.899633  92.720554      0.75765   
...                                      ...        ...          ...   
WSP.TO    2021-12-31               11.712991  90.131166      1.03480   
YUMC.K    2018-12-31               15.059853  66.837119      1.46973   
          2019-12-31               15.626857  72.437892      1.31285   
          2020-12-31               15.428441  85.913157      1.08237   
          2021-12-31               13.941668  88.809329      2.19545   

                       Scope 1  Scope 2 Location  Scope 2 Market  \
ticker    Dates                                                    
002555.SZ 2018-12-31     129.0            2701.0          2701.0   
          2019-12-31     129.0            2701.0          2701.0   
          2020-12-31     169.0            2395.0          2395.0   
          2021-12-31     152.0            1794.0          1794.0   
066570.KS 2018-12-31  785312.0          846510.0          5226.0   
...                        ...               ...             ...   
WSP.TO    2021-12-31   23969.0           22617.0         16303.0   
YUMC.K    2018-12-31  114720.0         1956072.0       1956072.0   
          2019-12-31  114720.0         1956072.0       1956072.0   
          2020-12-31  135615.0         1873757.0       1873757.0   
          2021-12-31  158887.0         2158856.0       2158856.0   

                      Expense Ratio  
ticker    Dates                      
002555.SZ 2018-12-31       0.509936  
          2019-12-31       0.817504  
          2020-12-31       1.040377  
          2021-12-31       1.054852  
066570.KS 2018-12-31       1.431860  
...                             ...  
WSP.TO    2021-12-31       0.944226  
YUMC.K    2018-12-31       1.630744  
          2019-12-31       1.621258  
          2020-12-31       1.133094  
          2021-12-31       0.671448  

[492 rows x 10 columns]

In [59]:
# Reorder Columns to have Expense Ratio as the 7th Column
cols=panel.columns.tolist()
cols=cols[:6]+cols[-1:]+cols[6:-1]
panel=panel[cols]
panel

Sector  Asset Turnover  Current Ratio  \
ticker    Dates                                                               
002555.SZ 2018-12-31  Communication Services        0.796177        2.60977   
          2019-12-31  Communication Services        0.869529        2.09920   
          2020-12-31  Communication Services        1.426797        2.44392   
          2021-12-31  Communication Services        1.390631        1.28937   
066570.KS 2018-12-31  Consumer Discretionary        1.552838        1.09457   
...                                      ...             ...            ...   
WSP.TO    2021-12-31             Industrails        1.005384        1.03480   
YUMC.K    2018-12-31  Consumer Discretionary        1.938857        1.75977   
          2019-12-31  Consumer Discretionary        1.891649        1.55786   
          2020-12-31  Consumer Discretionary        1.518339        1.30127   
          2021-12-31  Consumer Discretionary        0.927125        2.38800   

                      EBITDA Margin, Percent  ESG Score  Quick Ratio  \
ticker    Dates                                                        
002555.SZ 2018-12-31               27.457111  27.347583      2.54271   
          2019-12-31               23.891154  38.609209      2.09920   
          2020-12-31               20.995620  50.547773      2.44392   
          2021-12-31               21.787496  59.321713      1.28937   
066570.KS 2018-12-31                6.899633  92.720554      0.75765   
...                                      ...        ...          ...   
WSP.TO    2021-12-31               11.712991  90.131166      1.03480   
YUMC.K    2018-12-31               15.059853  66.837119      1.46973   
          2019-12-31               15.626857  72.437892      1.31285   
          2020-12-31               15.428441  85.913157      1.08237   
          2021-12-31               13.941668  88.809329      2.19545   

                      Expense Ratio   Scope 1  Scope 2 Location  \
ticker    Dates                                                   
002555.SZ 2018-12-31       0.509936     129.0            2701.0   
          2019-12-31       0.817504     129.0            2701.0   
          2020-12-31       1.040377     169.0            2395.0   
          2021-12-31       1.054852     152.0            1794.0   
066570.KS 2018-12-31       1.431860  785312.0          846510.0   
...                             ...       ...               ...   
WSP.TO    2021-12-31       0.944226   23969.0           22617.0   
YUMC.K    2018-12-31       1.630744  114720.0         1956072.0   
          2019-12-31       1.621258  114720.0         1956072.0   
          2020-12-31       1.133094  135615.0         1873757.0   
          2021-12-31       0.671448  158887.0         2158856.0   

                      Scope 2 Market  
ticker    Dates                       
002555.SZ 2018-12-31          2701.0  
          2019-12-31          2701.0  
          2020-12-31          2395.0  
          2021-12-31          1794.0  
066570.KS 2018-12-31          5226.0  
...                              ...  
WSP.TO    2021-12-31         16303.0  
YUMC.K    2018-12-31       1956072.0  
          2019-12-31       1956072.0  
          2020-12-31       1873757.0  
          2021-12-31       2158856.0  

[492 rows x 10 columns]

In [61]:
# Create new Scope 1 + 2 Variables
panel['Scope 1 + 2 Location']=panel['Scope 1']+panel['Scope 2 Location']
panel['Scope 1 + 2 Market']=panel['Scope 1']+panel['Scope 2 Market']

# Log Transformations
panel['Scope 1 Log']=np.log(panel['Scope 1'])
panel['Scope 2 Location Log']=np.log(panel['Scope 2 Location'])
panel['Scope 2 Market Log']=np.log(panel['Scope 2 Market'])
panel['Scope 1 + 2 Location Log']=np.log(panel['Scope 1 + 2 Location'])
panel['Scope 1 + 2 Market Log']=np.log(panel['Scope 1 + 2 Market'])

# Drop Original Scope 1 + 2 Variables
panel.drop(['Scope 1', 'Scope 2 Location', 'Scope 2 Market', 'Scope 1 + 2 Location', 'Scope 1 + 2 Market'], axis=1, inplace=True)

C:\Users\Evan Mc Garry\AppData\Local\Temp\ipykernel_20112\3968295293.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel['Scope 1 + 2 Location']=panel['Scope 1']+panel['Scope 2 Location']
C:\Users\Evan Mc Garry\AppData\Local\Temp\ipykernel_20112\3968295293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel['Scope 1 + 2 Market']=panel['Scope 1']+panel['Scope 2 Market']
C:\Users\Evan Mc Garry\AppData\Local\Temp\ipykernel_20112\3968295293.py:6: SettingWithCopyWarning: 
A value is trying to be set 

In [133]:
# Add Constant to Panel
panel = sm.add_constant(panel)

# Create Dummy Variables for Utilities Sector due to High Emissions Outliers
panel['Utilities'] = np.where(panel['Sector'] == 'Utilities', 1, 0)

# Reorder Columns to have Utilities as the 9th Column
cols=panel.columns.tolist()
cols=cols[:8]+cols[-1:]+cols[8:-1]
panel=panel[cols]

panel

const                  Sector  Asset Turnover  \
ticker    Dates                                                       
002555.SZ 2018-12-31    1.0  Communication Services        0.796177   
          2019-12-31    1.0  Communication Services        0.869529   
          2020-12-31    1.0  Communication Services        1.426797   
          2021-12-31    1.0  Communication Services        1.390631   
066570.KS 2018-12-31    1.0  Consumer Discretionary        1.552838   
...                     ...                     ...             ...   
WSP.TO    2021-12-31    1.0             Industrails        1.005384   
YUMC.K    2018-12-31    1.0  Consumer Discretionary        1.938857   
          2019-12-31    1.0  Consumer Discretionary        1.891649   
          2020-12-31    1.0  Consumer Discretionary        1.518339   
          2021-12-31    1.0  Consumer Discretionary        0.927125   

                      Current Ratio  EBITDA Margin, Percent  ESG Score  \
ticker    Dates                                                          
002555.SZ 2018-12-31        2.60977               27.457111  27.347583   
          2019-12-31        2.09920               23.891154  38.609209   
          2020-12-31        2.44392               20.995620  50.547773   
          2021-12-31        1.28937               21.787496  59.321713   
066570.KS 2018-12-31        1.09457                6.899633  92.720554   
...                             ...                     ...        ...   
WSP.TO    2021-12-31        1.03480               11.712991  90.131166   
YUMC.K    2018-12-31        1.75977               15.059853  66.837119   
          2019-12-31        1.55786               15.626857  72.437892   
          2020-12-31        1.30127               15.428441  85.913157   
          2021-12-31        2.38800               13.941668  88.809329   

                      Quick Ratio  Expense Ratio  Utilities  Scope 1 Log  \
ticker    Dates                                                            
002555.SZ 2018-12-31      2.54271       0.509936          0     4.859812   
          2019-12-31      2.09920       0.817504          0     4.859812   
          2020-12-31      2.44392       1.040377          0     5.129899   
          2021-12-31      1.28937       1.054852          0     5.023881   
066570.KS 2018-12-31      0.75765       1.431860          0    13.573836   
...                           ...            ...        ...          ...   
WSP.TO    2021-12-31      1.03480       0.944226          0    10.084517   
YUMC.K    2018-12-31      1.46973       1.630744          0    11.650250   
          2019-12-31      1.31285       1.621258          0    11.650250   
          2020-12-31      1.08237       1.133094          0    11.817575   
          2021-12-31      2.19545       0.671448          0    11.975949   

                      Scope 2 Location Log  Scope 2 Market Log  \
ticker    Dates                                                  
002555.SZ 2018-12-31              7.901377            7.901377   
          2019-12-31              7.901377            7.901377   
          2020-12-31              7.781139            7.781139   
          2021-12-31              7.492203            7.492203   
066570.KS 2018-12-31             13.648877            8.561401   
...                                    ...                 ...   
WSP.TO    2021-12-31             10.026457            9.699104   
YUMC.K    2018-12-31             14.486449           14.486449   
          2019-12-31             14.486449           14.486449   
          2020-12-31             14.443456           14.443456   
          2021-12-31             14.585089           14.585089   

                      Scope 1 + 2 Location Log  Scope 1 + 2 Market Log  
ticker    Dates                                                         
002555.SZ 2018-12-31                  7.948032                7.948032  
          2019-12-31                  7.948032                7.9480

In [134]:
# Prepare for Model Creation
import linearmodels as lm
from linearmodels import PanelOLS
X_vars = ['const', 'Asset Turnover', 'Current Ratio', 'EBITDA Margin, Percent', 'ESG Score', 'Quick Ratio', 'Expense Ratio', 'Utilities']
Y_vars = ['Scope 1 Log', 'Scope 2 Location Log', 'Scope 2 Market Log', 'Scope 1 + 2 Location Log', 'Scope 1 + 2 Market Log']

In [148]:
# Model 1 - PanelOLS Scope 1 (TIME-EFFECTS)
m1 = PanelOLS(dependent=panel['Scope 1 Log'], exog=panel[X_vars], time_effects=True, entity_effects=False)
r1 = m1.fit(cov_type='clustered', cluster_entity=True)
print(r1)

                          PanelOLS Estimation Summary                           
Dep. Variable:            Scope 1 Log   R-squared:                        0.2746
Estimator:                   PanelOLS   R-squared (Between):              0.2899
No. Observations:                 492   R-squared (Within):              -2.2866
Date:                Thu, Jun 22 2023   R-squared (Overall):              0.2729
Time:                        17:29:18   Log-likelihood                   -1100.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      26.017
Entities:                         123   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(7,481)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             8.1459
                            

In [145]:
# Model 1 - PanelOLS Scope 1 (ENTITY-EFFECTS)
m1_2 = PanelOLS(dependent=panel['Scope 1 Log'], exog=panel[X_vars], time_effects=False, entity_effects=True, drop_absorbed=True)
r1_2 = m1_2.fit(cov_type='clustered', cluster_entity=True)
print(r1_2)

## UNSUITABLE FOR USE

                          PanelOLS Estimation Summary                           
Dep. Variable:            Scope 1 Log   R-squared:                        0.0044
Estimator:                   PanelOLS   R-squared (Between):             -0.0023
No. Observations:                 492   R-squared (Within):               0.0044
Date:                Thu, Jun 22 2023   R-squared (Overall):             -0.0023
Time:                        17:28:37   Log-likelihood                    56.629
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.6167
Entities:                         123   P-value                           0.2044
Avg Obs:                       4.0000   Distribution:                   F(1,368)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):          5.933e+14
                            

C:\Users\Evan Mc Garry\AppData\Local\Temp\ipykernel_20112\3710237863.py:3: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Asset Turnover, Current Ratio, EBITDA Margin, Percent, Quick Ratio, Expense Ratio, Utilities

  r1_2 = m1_2.fit(cov_type='clustered', cluster_entity=True)


In [149]:
# Clean up Results
eff1 = r1.estimated_effects
eff1.reset_index(inplace=True)
eff1.rename(columns={'index':'ticker'}, inplace=True)
eff1 = eff1.drop(['ticker'], axis=1)
eff1 = eff1[0:4]
eff1.rename(columns={'estimated_effects':'Time Effects'}, inplace=True)
eff1

,Dates,Time Effects
0,2018-12-31,0.139600
1,2019-12-31,0.061513
2,2020-12-31,-0.041383
3,2021-12-31,-0.159730


In [136]:
# Model 2 - PanelOLS Scope 2 Location
m2 = PanelOLS(dependent=panel['Scope 2 Location Log'], exog=panel[X_vars], time_effects=True, entity_effects=False)
r2 = m2.fit(cov_type='clustered', cluster_entity=True)
print(r2)

                           PanelOLS Estimation Summary                            
Dep. Variable:     Scope 2 Location Log   R-squared:                        0.2091
Estimator:                     PanelOLS   R-squared (Between):              0.2344
No. Observations:                   492   R-squared (Within):              -1.9404
Date:                  Thu, Jun 22 2023   R-squared (Overall):              0.2050
Time:                          17:23:01   Log-likelihood                   -940.29
Cov. Estimator:               Clustered                                           
                                          F-statistic:                      18.169
Entities:                           123   P-value                           0.0000
Avg Obs:                         4.0000   Distribution:                   F(7,481)
Min Obs:                         4.0000                                           
Max Obs:                         4.0000   F-statistic (robust):             23.154
    

In [106]:
# Clean up Results
eff2 = r2.estimated_effects
eff2.reset_index(inplace=True)
eff2.rename(columns={'index':'ticker'}, inplace=True)
eff2 = eff2.drop(['ticker'], axis=1)
eff2 = eff2[0:4]
eff2.rename(columns={'estimated_effects':'Time Effects'}, inplace=True)
eff2

,Dates,Time Effects
0,2018-12-31,0.190422
1,2019-12-31,0.051050
2,2020-12-31,-0.045555
3,2021-12-31,-0.195917


In [137]:
# Model 3 - PanelOLS Scope 2 Market
m3 = PanelOLS(dependent=panel['Scope 2 Market Log'], exog=panel[X_vars], time_effects=True, entity_effects=False)
r3 = m3.fit(cov_type='clustered', cluster_entity=True)
print(r3)

                          PanelOLS Estimation Summary                           
Dep. Variable:     Scope 2 Market Log   R-squared:                        0.1728
Estimator:                   PanelOLS   R-squared (Between):              0.2169
No. Observations:                 492   R-squared (Within):              -0.3947
Date:                Thu, Jun 22 2023   R-squared (Overall):              0.1628
Time:                        17:23:15   Log-likelihood                   -1032.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      14.358
Entities:                         123   P-value                           0.0000
Avg Obs:                       4.0000   Distribution:                   F(7,481)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             6.9210
                            

In [113]:
# Clean up Results
eff3 = r3.estimated_effects
eff3.reset_index(inplace=True)
eff3.rename(columns={'index':'ticker'}, inplace=True)
eff3 = eff3.drop(['ticker'], axis=1)
eff3 = eff3[0:4]
eff3.rename(columns={'estimated_effects':'Time Effects'}, inplace=True)
eff3

,Dates,Time Effects
0,2018-12-31,0.368377
1,2019-12-31,0.151738
2,2020-12-31,-0.096780
3,2021-12-31,-0.423336


In [138]:
# Model 4 - PanelOLS Scope 1 + 2 Location
m4 = PanelOLS(dependent=panel['Scope 1 + 2 Location Log'], exog=panel[X_vars], time_effects=True, entity_effects=False)
r4 = m4.fit(cov_type='clustered', cluster_entity=True)
print(r4)

                             PanelOLS Estimation Summary                              
Dep. Variable:     Scope 1 + 2 Location Log   R-squared:                        0.2789
Estimator:                         PanelOLS   R-squared (Between):              0.2945
No. Observations:                       492   R-squared (Within):              -2.3998
Date:                      Thu, Jun 22 2023   R-squared (Overall):              0.2755
Time:                              17:23:28   Log-likelihood                   -972.06
Cov. Estimator:                   Clustered                                           
                                              F-statistic:                      26.578
Entities:                               123   P-value                           0.0000
Avg Obs:                             4.0000   Distribution:                   F(7,481)
Min Obs:                             4.0000                                           
Max Obs:                             4.0000

In [118]:
# Clean up Results
eff4 = r4.estimated_effects
eff4.reset_index(inplace=True)
eff4.rename(columns={'index':'ticker'}, inplace=True)
eff4 = eff4.drop(['ticker'], axis=1)
eff4 = eff4[0:4]
eff4.rename(columns={'estimated_effects':'Time Effects'}, inplace=True)
eff4

,Dates,Time Effects
0,2018-12-31,0.215717
1,2019-12-31,0.082766
2,2020-12-31,-0.051865
3,2021-12-31,-0.246617


In [139]:
# Model 5 - PanelOLS Scope 1 + 2 Market
m5 = PanelOLS(dependent=panel['Scope 1 + 2 Market Log'], exog=panel[X_vars], time_effects=True, entity_effects=False)
r5 = m5.fit(cov_type='clustered', cluster_entity=True)
print(r5)

                            PanelOLS Estimation Summary                             
Dep. Variable:     Scope 1 + 2 Market Log   R-squared:                        0.2706
Estimator:                       PanelOLS   R-squared (Between):              0.2895
No. Observations:                     492   R-squared (Within):              -1.9782
Date:                    Thu, Jun 22 2023   R-squared (Overall):              0.2653
Time:                            17:23:37   Log-likelihood                   -1010.7
Cov. Estimator:                 Clustered                                           
                                            F-statistic:                      25.490
Entities:                             123   P-value                           0.0000
Avg Obs:                           4.0000   Distribution:                   F(7,481)
Min Obs:                           4.0000                                           
Max Obs:                           4.0000   F-statistic (robust):

In [122]:
# Clean up Results
eff5 = r5.estimated_effects
eff5.reset_index(inplace=True)
eff5.rename(columns={'index':'ticker'}, inplace=True)
eff5 = eff5.drop(['ticker'], axis=1)
eff5 = eff5[0:4]
eff5.rename(columns={'estimated_effects':'Time Effects'}, inplace=True)
eff5

,Dates,Time Effects
0,2018-12-31,0.276626
1,2019-12-31,0.113394
2,2020-12-31,-0.055851
3,2021-12-31,-0.334169


In [155]:
# Compare 5 PanelOLS Models
from linearmodels.panel import compare

# Compare first 3 models
comp1 = compare({'Model 1':r1, 'Model 2':r2, 'Model 3':r3}, stars=True)
print(comp1.summary.as_latex())

\begin{center}
\begin{tabular}{lccc}
\toprule
                                    & \textbf{Model 1} &     \textbf{Model 2}     &    \textbf{Model 3}     \\
\midrule
\textbf{Dep. Variable}              &   Scope 1 Log    &   Scope 2 Location Log   &   Scope 2 Market Log    \\
\textbf{Estimator}                  &     PanelOLS     &         PanelOLS         &        PanelOLS         \\
\textbf{No. Observations}           &       492        &           492            &          492            \\
\textbf{Cov. Est.}                  &    Clustered     &        Clustered         &       Clustered         \\
\textbf{R-squared}                  &      0.2746      &          0.2091          &         0.1728          \\
\textbf{R-Squared (Within)}         &     -2.2866      &         -1.9404          &        -0.3947          \\
\textbf{R-Squared (Between)}        &      0.2899      &          0.2344          &         0.2169          \\
\textbf{R-Squared (Overall)}        &      0.2729      & 

In [156]:
# Compare last 2 models
comp2 = compare({'Model 4':r4, 'Model 5':r5}, stars=True)
print(comp2.summary.as_latex())

\begin{center}
\begin{tabular}{lcc}
\toprule
                                    &       \textbf{Model 4}       &      \textbf{Model 5}       \\
\midrule
\textbf{Dep. Variable}              &   Scope 1 + 2 Location Log   &   Scope 1 + 2 Market Log    \\
\textbf{Estimator}                  &           PanelOLS           &          PanelOLS           \\
\textbf{No. Observations}           &             492              &            492              \\
\textbf{Cov. Est.}                  &          Clustered           &         Clustered           \\
\textbf{R-squared}                  &            0.2789            &           0.2706            \\
\textbf{R-Squared (Within)}         &           -2.3998            &          -1.9782            \\
\textbf{R-Squared (Between)}        &            0.2945            &           0.2895            \\
\textbf{R-Squared (Overall)}        &            0.2755            &           0.2653            \\
\textbf{F-statistic}                &         

In [158]:
# Examine Multicollinearity

# Variance Inflation Factors
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(panel[X_vars].values, i) for i in range(panel[X_vars].shape[1])]
vif["features"] = panel[X_vars].columns
print(vif.round(1).to_latex(index=False))

\begin{tabular}{rl}
\toprule
 VIF Factor &               features \\
\midrule
       12.9 &                  const \\
       29.0 &         Asset Turnover \\
        4.7 &          Current Ratio \\
        1.6 & EBITDA Margin, Percent \\
        2.6 &              ESG Score \\
        4.6 &            Quick Ratio \\
       30.6 &          Expense Ratio \\
        1.2 &              Utilities \\
\bottomrule
\end{tabular}



C:\Users\Evan Mc Garry\AppData\Local\Temp\ipykernel_20112\3011979630.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(vif.round(1).to_latex(index=False))
